# 과적합 피하기 - 초음파광물 사례

In [1]:
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [5]:
import pandas as pd

df = pd.read_csv('dataset/sonar.csv', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [6]:
# seed 값 설정
seed = 0
np.random.seed(seed)
tf.set_random_seed(seed)

In [7]:
# 데이터 입력
dataset = df.values
x_data = np.array(dataset[:,0:60], dtype=np.float64)
Y_obj = dataset[:,60]

In [8]:
# 문자열 변환
from sklearn.preprocessing import LabelEncoder

e = LabelEncoder().fit(Y_obj)
y_data = np.array(e.transform(Y_obj), dtype=np.float64).reshape(208,1)
y_data[:5]

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.]])

In [9]:
x_data.shape, y_data.shape

((208, 60), (208, 1))

In [10]:
# 입력 값을 플레이스 홀더에 저장
X = tf.placeholder(tf.float64, shape=[None, 60])
Y = tf.placeholder(tf.float64, shape=[None, 1])

In [11]:
# Layer 1
W1 = tf.Variable(tf.random_normal([60,24], dtype=tf.float64)) 
b1 = tf.Variable(tf.random_normal([24], dtype=tf.float64))
L1 = tf.nn.relu(tf.add(tf.matmul(X, W1), b1))

In [12]:
# Layer 2
W2 = tf.Variable(tf.random_normal([24,10], dtype=tf.float64)) 
b2 = tf.Variable(tf.random_normal([10], dtype=tf.float64))
L2 = tf.nn.relu(tf.add(tf.matmul(L1, W2), b2))

In [13]:
# Layer 3
W3 = tf.Variable(tf.random_normal([10,1], dtype=tf.float64)) 
b3 = tf.Variable(tf.random_normal([1], dtype=tf.float64))
hypo = tf.sigmoid(tf.add(tf.matmul(L2, W3), b3))

In [14]:
# Binary cross entropy
cost = tf.reduce_sum(-Y * tf.log(hypo) - (1.-Y) * tf.log(1.-hypo))

In [15]:
learning_rate = 0.1
train = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [16]:
# 학습
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(261):
    _, cost_val = sess.run([train, cost], feed_dict = {X: x_data, Y: y_data})
    if epoch % 20 == 0:
        print("step=%d, \tcost_val=%.5f" % (epoch + 1, cost_val))

step=1, 	cost_val=791.61274
step=21, 	cost_val=128.99900
step=41, 	cost_val=98.03927
step=61, 	cost_val=81.56665
step=81, 	cost_val=70.62200
step=101, 	cost_val=63.88345
step=121, 	cost_val=56.09760
step=141, 	cost_val=49.51959
step=161, 	cost_val=58.71998
step=181, 	cost_val=37.53481
step=201, 	cost_val=32.08079
step=221, 	cost_val=21.63793
step=241, 	cost_val=15.82335
step=261, 	cost_val=11.32764


In [17]:
predict = tf.cast(hypo > 0.5, dtype=tf.float64)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predict, Y), dtype=tf.float64))

In [18]:
acc = sess.run(accuracy, feed_dict = {X: x_data, Y: y_data})
print("Accuracy: %.4f" % acc)

Accuracy: 0.9808


In [7]:
# 모델 설정
model = Sequential()
model.add(Dense(24, input_dim=60, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 24)                1464      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                250       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 1,725
Trainable params: 1,725
Non-trainable params: 0
_________________________________________________________________


In [9]:
# 모델 컴파일 
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [10]:
# 모델 실행
model.fit(X, Y, epochs=200, batch_size=5)

W0812 09:22:35.676828  5564 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0812 09:22:35.747832  5564 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Epoch 1/200
208/208 [==============================] - 0s 1ms/step - loss: 0.2540 - acc: 0.4567
Epoch 2/200
208/208 [==============================] - 0s 144us/step - loss: 0.2459 - acc: 0.5962
Epoch 3/200
208/208 [==============================] - 0s 144us/step - loss: 0.2413 - acc: 0.6394
Epoch 4/200
208/208 [==============================] - 0s 144us/step - loss: 0.2365 - acc: 0.6587
Epoch 5/200
208/208 [==============================] - 0s 149us/step - loss: 0.2309 - acc: 0.6827
Epoch 6/200
208/208 [==============================] - 0s 149us/step - loss: 0.2247 - acc: 0.7019
Epoch 7/200
208/208 [==============================] - 0s 139us/step - loss: 0.2183 - acc: 0.7067
Epoch 8/200
208/208 [==============================] - 0s 139us/step - loss: 0.2105 - acc: 0.7067
Epoch 9/200
208/208 [==============================] - 0s 139us/step - loss: 0.2026 - acc: 0.7260
Epoch 10/200
208/208 [==============================] - 0s 139us/step - loss: 0.1909 - acc: 0.7548
Epoch 11/200
208/208 

208/208 [==============================] - 0s 135us/step - loss: 7.6719e-04 - acc: 1.0000
Epoch 165/200
208/208 [==============================] - 0s 120us/step - loss: 7.8324e-04 - acc: 1.0000
Epoch 166/200
208/208 [==============================] - 0s 130us/step - loss: 6.8501e-04 - acc: 1.0000
Epoch 167/200
208/208 [==============================] - 0s 144us/step - loss: 8.0314e-04 - acc: 1.0000
Epoch 168/200
208/208 [==============================] - 0s 202us/step - loss: 6.6485e-04 - acc: 1.0000
Epoch 169/200
208/208 [==============================] - 0s 269us/step - loss: 6.9722e-04 - acc: 1.0000
Epoch 170/200
208/208 [==============================] - 0s 202us/step - loss: 6.2231e-04 - acc: 1.0000
Epoch 171/200
208/208 [==============================] - 0s 197us/step - loss: 6.1840e-04 - acc: 1.0000
Epoch 172/200
208/208 [==============================] - 0s 212us/step - loss: 5.4688e-04 - acc: 1.0000
Epoch 173/200
208/208 [==============================] - 0s 216us/step - loss:

In [11]:
# 결과 출력 
print("\n Accuracy: %.4f" % (model.evaluate(X, Y)[1]))

208/208 [==============================] - 0s 186us/step

 Accuracy: 1.0000
